In [1]:
import pysam

In [ ]:
PATH_MM_BAM     = ''
PATH_ALIGNED_BAM = ''
PATH_OUT_BAM    = ''
PATH_log        = ''

MM_BAM      = pysam.AlignmentFile(PATH_MM_BAM, 'rb')
ALIGNED_BAM = pysam.AlignmentFile(PATH_ALIGNED_BAM, 'rb')
OUT_BAM     = pysam.AlignmentFile(PATH_OUT_BAM, 'wb', template=ALIGNED_BAM)
log         = open(PATH_log, 'w')

In [ ]:
# [1] Build dictionary
dict_ReadName_to_Tags = dict()

for read in MM_BAM.fetch(until_eof=True):
    try:
        ML_tag = read.get_tag('ML')
        MM_tag = read.get_tag('MM')
    except: 
        continue 
    
    dict_ReadName_to_Tags[read.query_name] = [ ML_tag, MM_tag ]

# [2] Tag MM/ML tags
for read in ALIGNED_BAM.fetch():
    if read.is_secondary==True or read.is_supplementary==True: continue 
    
    try:
        ML_tag, MM_tag = dict_ReadName_to_Tags[read.query_name][0], dict_ReadName_to_Tags[read.query_name][1]
    except KeyError:
        log.write(f'{read.query_name}\n')
        
    read.set_tag('ML', ML_tag)
    read.set_tag('MM', MM_tag)
    OUT_BAM.write(read)

MM_BAM.close()
ALIGNED_BAM.close()
OUT_BAM.close()
log.close()